In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

cuda


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
# encoders and decoders = tokenizer
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [4]:
# training and validation splits
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data= train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    print(ix)
    print(ix.shape)    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print(f'\nsize x matrix {x.shape}\n')
print('inputs: ')
print(x)
print('targets: ')
print(y)

tensor([166189,  65724,  32362,  93114])
torch.Size([4])

size x matrix torch.Size([4, 8])

inputs: 
tensor([[73,  1, 73, 54, 64, 58,  0, 65],
        [58, 58, 69,  9,  3,  1, 72, 54],
        [74, 71,  1, 74, 67, 76, 58, 65],
        [62, 68, 67, 11,  0,  0,  3, 47]], device='cuda:0')
targets: 
tensor([[ 1, 73, 54, 64, 58,  0, 65, 68],
        [58, 69,  9,  3,  1, 72, 54, 62],
        [71,  1, 74, 67, 76, 58, 65, 56],
        [68, 67, 11,  0,  0,  3, 47, 61]], device='cuda:0')


In [5]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print('when input is', context, 'target is ', target)

# how to measure the loss:
# -ln(1/80) = to predict the next carachter. // have to minimize the loss, the objective
# utilize adamW as the optimize algorithm

In [12]:
class BigramLanguageModel(nn.Module):

    def __init__ (self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits =  self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the laste time step
            logits = logits[:, -1, :] # becomes(B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



&3h]9P31kbq.t-u5him_ajn!WE2;a.
jlXF,G-D&!QoyCd_Ylr3nNz(za.;sVrJY4Hv9y5m-?4fzY&k;wSt4fll1WulXnvJ5!G-(zy18Kx(z2DQo 3rQ!8 1Rh3rV2;H,4
GZU2OSl1(gP7' jF'3rOYwr.E!;.inRBM:X2vrPnLoCt-[gYu0 6ukIrTrC'nhu)Vq5]:xMEjl]S1yE[S!L0FI,iZw!R30F!FKprJz,rJu!Lf:ecUwBopPbH,cLg[sAZXi-POEUOq
85*!(

S]ig8l(z;YVB h,
BWk_op.iW[dK,)
isHH:Unj5l6UQyEXx&po:eswEHt_kUB*,j1
S-6YZUZ1O)N?aMdCqYD;I;IrxKXS-[in-0YM_hmgxKnlkJ0k_lf9Q'VHlSTTqXWnUd*R(;0vSsUZ4mdY&FjFh4pFD.iCy-v1:Rh"]0*vhPRoyHrO9IPvC9oC)V]l_iKxuJevm[gGqoX(5Ej:UAs66Q:[Be(U2


In [13]:
torch.zeros(1,1)

tensor([[0.]])